## Mood Based Library Assistant

RAG pipeline with OpenAI + Chroma. Conversational: history is used; a **query rewriter** (GPT-4.1-nano) rewrites the current question using the **last 3 user questions** for better retrieval (handles "something like that", "more like the last one", etc.). Retrieval uses a **score threshold** so only high-quality contexts are returned; fallback to standard retrieval if none pass. Book availability and user-rating info are appended from the tool.


Run all cells in order. Ensure you are in the notebook's directory (so `knowledge_base/` and this notebook are in the same folder) and that `OPENAI_API_KEY` is set in `.env`.

In [ ]:
# Imports and paths
import os
import random
import sqlite3
from pathlib import Path

from dotenv import load_dotenv
import pandas as pd
import gradio as gr

from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.tools import tool

load_dotenv(override=True)

# Knowledge base folder is next to this notebook (same parent directory)
BASE_DIR = Path.cwd()
KB_DIR = BASE_DIR / "knowledge_base"
CHROMA_DIR = BASE_DIR / "vector_db"
DB_PATH = BASE_DIR / "library.db"

print("KB_DIR:", KB_DIR)
print("CHROMA_DIR:", CHROMA_DIR)
print("DB_PATH:", DB_PATH)

In [ ]:
# Load all CSVs from knowledge_base; each row = one Document with full row text + metadata
def load_csv_documents():
    documents = []
    if not KB_DIR.exists():
        raise FileNotFoundError(f"Knowledge base folder not found: {KB_DIR}")
    for csv_path in sorted(KB_DIR.glob("*.csv")):
        df = pd.read_csv(csv_path, on_bad_lines='skip')
        for _, row in df.iterrows():
            # Full row as readable text for embedding/search
            parts = [f"{col}: {row[col]}" for col in df.columns if pd.notna(row.get(col))]
            page_content = "\n".join(parts)
            if not page_content.strip():
                continue
            metadata = {"source": csv_path.name}
            for col in df.columns:
                val = row.get(col)
                if pd.notna(val) and isinstance(val, str) and len(str(val)) < 500:
                    metadata[col] = str(val)
            documents.append(Document(page_content=page_content, metadata=metadata))
    return documents

documents = load_csv_documents()
print(f"Loaded {len(documents)} document chunks (one per CSV row).")

In [ ]:
# Vector store: Chroma with OpenAI embeddings (chunks = full CSV rows with metadata)
EMBEDDING_MODEL = "text-embedding-3-small"
LLM_MODEL = "gpt-4o-mini"

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
CHROMA_DIR.mkdir(parents=True, exist_ok=True)
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=str(CHROMA_DIR),
)
# Retriever with score threshold so only high-quality contexts are returned (Chroma uses similarity, higher = better)
SCORE_THRESHOLD = 0.8
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 5, "score_threshold": SCORE_THRESHOLD},
)
print("Chroma vector store ready (with score threshold).")

In [ ]:
# SQLite: books table (title, author, availability, rating_count, rating_summary)
RATING_SUMMARIES = [
    "readers rated it a good read",
    "people found it worth the time",
    "readers would recommend it",
    "of readers enjoyed it",
    "readers said it was a page-turner",
    "of readers found the book boring"
]

def init_availability_db():
    conn = sqlite3.connect(str(DB_PATH))
    conn.execute("""
        CREATE TABLE IF NOT EXISTS books (
            title TEXT NOT NULL,
            author TEXT NOT NULL,
            availability TEXT NOT NULL,
            rating_count INTEGER DEFAULT 0,
            rating_summary TEXT,
            PRIMARY KEY (title, author)
        )
    """)
    conn.commit()
    books_seen = set()
    for doc in documents:
        t, a = doc.metadata.get("title"), doc.metadata.get("author")
        if t and a and (t, a) not in books_seen:
            books_seen.add((t, a))
    book_list = list(books_seen)
    if not book_list:
        conn.close()
        return
    n = min(random.randint(5, max(5, len(book_list))), len(book_list))
    chosen = random.sample(book_list, n)
    statuses = ["available", "not available", "checked out", "on hold"]
    for (title, author) in chosen:
        status = random.choice(statuses)
        count = random.randint(20, 400)
        summary = f"{count} {random.choice(RATING_SUMMARIES)}"
        conn.execute(
            """INSERT OR REPLACE INTO books (title, author, availability, rating_count, rating_summary)
               VALUES (?, ?, ?, ?, ?)""",
            (title, author, status, count, summary),
        )
    conn.commit()
    conn.close()

init_availability_db()
print("SQLite library.db initialized with availability and user-rating data.")

In [ ]:
# Tool: get book info (availability + user ratings). No result = no record = clear (no extra note).
@tool
def get_book_info(title: str, author: str = "") -> str:
    """Get library info for a book: availability and user ratings. Pass title; author is optional (use title-only to avoid matching books when only the author is mentioned)."""
    conn = sqlite3.connect(str(DB_PATH))
    if author:
        row = conn.execute(
            "SELECT availability, rating_count, rating_summary FROM books WHERE title = ? AND author = ?",
            (title.strip(), author.strip()),
        ).fetchone()
    else:
        row = conn.execute(
            "SELECT availability, rating_count, rating_summary FROM books WHERE title = ? LIMIT 1",
            (title.strip(),),
        ).fetchone()
    conn.close()
    if row:
        availability, count, summary = row[0], row[1], row[2] or ""
        parts = [f"Availability: {availability}"]
        if summary:
            parts.append(summary)
        elif count:
            parts.append(f"{count} readers gave feedback.")
        return ". ".join(parts)
    return "No record in library system (assume clear/available)."

def get_book_info_for_response(response_text: str, docs) -> str:
    """Call the tool only for books whose *title* appears in the final response (title-only lookup; no author to avoid irrelevant matches)."""
    if not response_text or not docs:
        return ""
    # Unique titles from retrieved context (candidates)
    candidate_titles = set()
    for doc in docs:
        t = doc.metadata.get("title")
        if t and t.strip():
            candidate_titles.add(t.strip())
    # Only call tool for titles that appear in the assistant's final response
    lines = []
    for title in sorted(candidate_titles):
        if title not in response_text:
            continue
        result = get_book_info.invoke({"title": title, "author": ""})
        lines.append(f"- **{title}**: {result}")
    if not lines:
        return ""
    return "\n\n**Book info (availability & ratings):**\n" + "\n".join(lines)

In [ ]:
# Query rewriter: use full conversation (user + assistant) so "they" / "that author" can be resolved
REWRITER_MODEL = "gpt-4.1-nano"  # or "gpt-4o-mini" if 4.1-nano is not available
rewriter_llm = ChatOpenAI(model=REWRITER_MODEL, temperature=0)

REWRITER_PROMPT = """You are a query rewriter for a library book search. Turn the current user question into a single, self-contained search query that will find the right books.

Use the recent conversation (both user and assistant) to resolve references:
- "they" / "that author" / "the same writer" → use the author from the assistant's previous answer (e.g. User: "Who is the author of Dune?" Assistant: "Frank Herbert." User: "What other books do they have?" → rewrite as "other books by Frank Herbert").
- "that book" / "like the last one" / "something similar" → use the book or theme from the previous turn.
Including the assistant's answers is essential so you know what "they" or "that book" refers to. Output only the rewritten query, no explanation.

Recent conversation (oldest first):
{conversation}

Current user question: {current_question}

Rewritten search query:"""

def _format_conversation_for_rewriter(history, max_turns=2):
    """Format last N turns as 'User: ... Assistant: ...' so the rewriter can resolve pronouns."""
    if not history:
        return "(none)"
    turns = []
    for turn in history[-max_turns:]:
        if hasattr(turn, "content") and hasattr(turn, "role"):
            role = getattr(turn, "role", "")
            content = (getattr(turn, "content", "") or "").strip()
            if role == "user":
                turns.append(f"User: {content}")
            elif role == "assistant":
                turns.append(f"Assistant: {content}")
        elif isinstance(turn, (list, tuple)) and len(turn) >= 2:
            if turn[0]:
                turns.append(f"User: {turn[0]}")
            if turn[1]:
                turns.append(f"Assistant: {turn[1]}")
    return "\n".join(turns) if turns else "(none)"

def rewrite_query(current_question: str, history: list) -> str:
    """Rewrite the current question using full conversation (user + assistant) for retrieval."""
    conversation = _format_conversation_for_rewriter(history, max_turns=3)
    prompt = REWRITER_PROMPT.format(conversation=conversation, current_question=current_question)
    response = rewriter_llm.invoke([HumanMessage(content=prompt)])
    rewritten = (response.content or current_question).strip()
    return rewritten if rewritten else current_question

In [ ]:
# Main LLM and RAG prompt (conversational: history is used in messages)
llm = ChatOpenAI(model=LLM_MODEL, temperature=0, streaming=True)

SYSTEM_PROMPT_TEMPLATE = """You are a friendly library assistant. Use the following context from the library catalog to answer the user. Recommend books when relevant (by title and author). If you don't know, say so. Be concise and helpful. You have access to the conversation history.

Context:
{context}
"""

In [ ]:
# Retrieve with scores; dedupe once so LLM context and displayed chunks always match. Chroma returns distance (lower=better)
def _dedupe_docs(docs, scores):
    """Remove duplicate docs by (title, author) so same book from multiple CSVs counts once. Keep first (best score)."""
    seen = set()
    out_docs, out_scores = [], []
    for doc, score in zip(docs, scores):
        key = (str(doc.metadata.get("title") or "").strip(), str(doc.metadata.get("author") or "").strip())
        if key in seen:
            continue
        seen.add(key)
        out_docs.append(doc)
        out_scores.append(score)
    return out_docs, out_scores

def _retrieve_with_scores(query: str):
    """Returns (deduplicated docs, scores). Single source for LLM context, display, and book-info tool."""
    try:
        pairs = vectorstore.similarity_search_with_relevance_scores(query, k=10)
    except Exception:
        pairs = [(d, 0.0) for d in vectorstore.similarity_search(query, k=10)]
    if not pairs:
        return [], []
    docs, scores = zip(*pairs)
    docs, scores = _dedupe_docs(list(docs), list(scores))
    docs, scores = docs[:5], scores[:5]
    return docs, scores

def format_context_with_scores(docs, scores) -> str:
    """Format retrieved chunks as markdown. Chroma returns distance (lower=better); show as similarity (higher=better)."""
    if not docs:
        return "_No retrieved context._"
    lines = []
    for i, (doc, raw) in enumerate(zip(docs, scores), 1):
        if isinstance(raw, (int, float)):
            similarity = 1.0 - raw if raw <= 1.0 else 1.0 / (1.0 + raw)
            score_str = f"similarity: `{similarity:.3f}` (distance: {raw:.3f})"
        else:
            score_str = str(raw)
        lines.append(f"**Chunk {i}** | {score_str}")
        lines.append("")
        lines.append(doc.page_content.replace("\n", "  \n"))
        lines.append("")
        lines.append("---")
        lines.append("")
    return "\n".join(lines).rstrip()

# Chat: one deduplicated docs list for LLM context, display, and book-info (keeps counts consistent)
def chat_with_availability(message, history):
    text = getattr(message, "content", message) or message
    text = str(text or "").strip()
    if not text:
        yield "", ""
        return
    rewritten = rewrite_query(text, history)
    docs, scores = _retrieve_with_scores(rewritten)
    context = "\n\n".join(doc.page_content for doc in docs) if docs else "(No closely matching books found.)"
    context_md = format_context_with_scores(docs, scores)
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    messages = [SystemMessage(content=system_prompt)]
    for turn in history:
        if hasattr(turn, "content") and hasattr(turn, "role"):
            role, content = getattr(turn, "role", ""), getattr(turn, "content", "") or ""
            if role == "user":
                messages.append(HumanMessage(content=content))
            elif role == "assistant":
                messages.append(AIMessage(content=content))
        elif isinstance(turn, (list, tuple)) and len(turn) >= 2:
            if turn[0]:
                messages.append(HumanMessage(content=str(turn[0])))
            if turn[1]:
                messages.append(AIMessage(content=str(turn[1])))
    messages.append(HumanMessage(content=text))
    full = ""
    for chunk in llm.stream(messages):
        if chunk.content:
            full += chunk.content
            yield full, ""
    extra = get_book_info_for_response(full, docs)
    if extra:
        full += extra
    yield full, context_md

In [ ]:
# Gradio: chat + second output (retrieved context with scores) in a toggleable Accordion
with gr.Blocks(title="Mood Based Library Assistant", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Mood Based Library Assistant")
    gr.Markdown("Ask for book recommendations by mood, theme, or topic. You'll get availability and user-rating info for recommended books.")
    with gr.Accordion("Retrieved context (toggle to view)", open=False):
        context_out = gr.Markdown(value="", label="Context with scores")
    chat_if = gr.ChatInterface(
        fn=chat_with_availability,
        type="messages",
        additional_outputs=[context_out],
    )
demo.launch(inbrowser=True)